Analysis of Simulation Studies
----------
Author: Albert Ulmer  
Date: 2022-06-04 - 2022-06-30

In [ ]:
# autoreload packages
%load_ext autoreload
%autoreload 2

# import data & plotting libraries
import numpy as np
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
plt.style.use('default')

from matplotlib import cm
import sqlite3

# import own libraries
import util
import plot
import config




In [ ]:
### Set global flag whether to save plots to files or not
writefiles = 1

if writefiles:
    print("Writing output files!")
else:
    print("Leaving files alone!")


In [ ]:
### Set global flag whether to print debug messages while running code
showdebug = 1

if showdebug:
    print("Showing debug messages!")
else:
    print("No debug messages will be shown!")

# Load Simulation Data


In [ ]:
# connect to SQLite database
try:
    conn = sqlite3.connect('./database_gurobi/dsmdata-sim.sqlite3')
    conn_cbc = sqlite3.connect('./database_cbc/dsmdata-sim.sqlite3')
    conn_cbc_3200u = sqlite3.connect('./database_cbc_3200u/dsmdata-sim.sqlite3')

    print('Connected to database...')
except:
    print('Database error!')
    exit()

In [ ]:
# execute SQL query
queryps = open("sqls/model_griddraw_comptime_v2.sql").read()
dfps = pd.read_sql_query(queryps, conn)
dfps.pvprc = dfps.pvprc*100
dfps.bessprc = dfps.bessprc*100
dfps.SSR = dfps.SSR*100
dfps.SCR = dfps.SCR*100
if showdebug: print(dfps.tail())


In [ ]:
# execute SQL query
queryps = open("sqls/model_griddraw_comptime_v1.sql").read()
dfps_cbc = pd.read_sql_query(queryps, conn_cbc)
dfps_cbc.pvprc = dfps_cbc.pvprc*100
dfps_cbc.bessprc = dfps_cbc.bessprc*100
if showdebug: print(dfps_cbc.tail())

In [ ]:
# execute SQL query
queryps = open("sqls/model_griddraw_comptime_v1.sql").read()
dfps_cbc_3200u = pd.read_sql_query(queryps, conn_cbc_3200u)
dfps_cbc_3200u.pvprc = dfps_cbc_3200u.pvprc*100
dfps_cbc_3200u.bessprc = dfps_cbc_3200u.bessprc*100
if showdebug: print(dfps_cbc_3200u.tail())

# Analysis

## Peakshaving

### PAPR & Standard Deviation

In [ ]:
dfps_pivot = dfps.pivot_table(values="GridDraw", index=["pvprc", "bessprc"], columns="model", aggfunc=["mean", "std", "max", util.papr])
if showdebug: print(dfps_pivot.head())

In [ ]:
# Translation dictionaries for plotting
model_desc = {}
model_desc["direct"] = "Direct Charging"
model_desc["rule"] = "Rule-based Charging"
model_desc["pred"] = "Predictive Charging"
model_desc["stoch"] = "Stochastic Charging"
model_desc["perfect"] = "Perfect Information"

model_abbr = {}
model_abbr["direct"] = "Direct"
model_abbr["rule"] = "Rule-based"
model_abbr["pred"] = "Predictive"
model_abbr["stoch"] = "Stochastic"
model_abbr["perfect"] = "Perfect"

metric_desc = {}
metric_desc["max"] = "Maximum Grid Draw [kW]"
metric_desc["mean"] = "Average Grid Draw [kW]"
metric_desc["papr"] = "Peak-to-Average Power Ratio"
metric_desc["std"] = "Standard Deviation"

variable_desc = {}
variable_desc["pvprc"] = "PV size [%]"
variable_desc["bessprc"] = "BESS size [%]"

In [ ]:
models = ['direct', 'rule', 'pred', 'stoch', 'perfect']
metrics = ['max', 'mean', 'std', 'papr']


In [ ]:
for model in models: #list(model_desc.keys()):
    for metric in metrics: #list(metric_desc.keys()):
        if showdebug: print("Working on model", model, "and metric", metric)
        label = model + "_" + metric
        my_data = np.array(dfps_pivot.loc[:,(metric)].reset_index()[["pvprc", "bessprc", model]])
        if my_data[:,2].std() == 0:
            myplot = plot.contour_plot(mydata = my_data, order = 0, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel = metric_desc[metric]) #, title = model_desc[model])
        else:
            myplot = plot.contour_plot(mydata = my_data, order = 2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel = metric_desc[metric]) #, title = model_desc[model])
        if writefiles:
            myplot.savefig("output/peakshaving_contour_"+label+".png",
                           bbox_inches='tight', dpi=300)
            plt.close(myplot)
        #if model in  ["direct", "rule"] and metric == "max":
        if my_data[:,2].std() == 0:
            mysurf = plot.surface_plot(mydata = my_data, order = 0, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel = metric_desc[metric]) #, title = model_desc[model])
        else:
            mysurf = plot.surface_plot(mydata = my_data, order = 2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel = metric_desc[metric]) #, title = model_desc[model])
        if writefiles:
            mysurf.savefig("output/peakshaving_surface_"+label+".png",
                           bbox_inches='tight', dpi=300)
            plt.close(mysurf)

In [ ]:
dfps_pivot.round(2)


In [ ]:
dfps_pivot_pretty = dfps_pivot.round(2).copy()
dfps_pivot_pretty.index.names = ["PV %", "BESS %"]
dfps_pivot_pretty.columns.set_levels(["Average", "Std. Dev.", "Maximum", "PAPR"], level=0, inplace=True)
dfps_pivot_pretty.columns.set_levels(["Direct", "Perfect", "Predictive", "Rule-based", "Stochastic"], level=1, inplace=True)
#dfps_pivot_pretty.to_latex(buf="output/peakshaving_results.tex", bold_rows=True)
if writefiles:
    dfps_pivot_pretty.loc[:, ["Maximum", "Average"]].to_latex(
        buf="output/peakshaving_results1.tex", bold_rows=True)
    dfps_pivot_pretty.loc[:, ["Std. Dev.", "PAPR"]].to_latex(
        buf="output/peakshaving_results2.tex", bold_rows=True)


In [ ]:
list(model_abbr.keys())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfps, x="model", y="GridDraw", order=list(model_abbr.keys()),
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Grid Draw [kW]")
ax.set_xticklabels(list(model_abbr.values()))

if writefiles:
    plt.savefig("output/GridDraw_violin.png",
                bbox_inches='tight', dpi=300)
    #plt.close()

## Computation Time

### Gurobi

In [ ]:
dfrt_pivot = dfps.pivot_table(values="runningtime",  index="model", aggfunc=["mean", "std", "max"]).round(3)
if showdebug: print(dfrt_pivot.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

# models = ["Direct", "Rule-based", "Predictive", "Stochastic", "Perfect"]

f, ax = plt.subplots()
sns.violinplot(data=dfps, x="model", y="runningtime", order=list(model_abbr.keys()),
               alpha=1, bw=.25, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Computation Time [s]")
ax.set_xticklabels(list(model_abbr.values()))
ax.set_yscale("log")

if writefiles:
    plt.savefig("output/runningtime_violin.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


### CBC


In [ ]:
dfrt_pivot_cbc = dfps_cbc.pivot_table(values="runningtime",  index="model", aggfunc=["mean", "std", "max"]).round(3)
if showdebug: print(dfrt_pivot_cbc.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfps_cbc, x="model", y="runningtime", order=list(model_abbr.keys()),
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Computation Time [s]")
ax.set_xticklabels(list(model_abbr.values()))
ax.set_yscale("log")

if writefiles:
    plt.savefig("output/runningtime_violin_cbc.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


### CBC on Ryzen 3 3200U

In [ ]:
dfrt_pivot_cbc_3200u = dfps_cbc_3200u.pivot_table(values="runningtime",  index="model", aggfunc=["mean", "std", "max"]).round(3)
if showdebug: print(dfrt_pivot_cbc_3200u.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfps_cbc_3200u, x="model", y="runningtime", order=list(model_abbr.keys()),
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Computation Time [s]")
ax.set_xticklabels(list(model_abbr.values()))
ax.set_yscale("log")

if writefiles:
    plt.savefig("output/runningtime_violin_cbc_3200u.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


In [ ]:
dfrt_pivot.merge(dfrt_pivot_cbc, left_index=True, right_index=True, suffixes=["_i7-Gurobi", "_i7-CBC"]).round(3)

In [ ]:
dfps["env"] = "Gurobi on i7"
dfps_cbc["env"] = "CBC on i7"
dfps_cbc_3200u["env"] = "CBC on 3200U"

dfrt_all = pd.concat([dfps, dfps_cbc, dfps_cbc_3200u])
dfrt_all

dfrt_pivot_all = dfrt_all.pivot_table(values="runningtime",  index="model", columns=["env"], aggfunc=["mean", "std", "max"]).round(3)
dfrt_pivot_all.columns.names = ["Metric", "CPU"]
dfrt_pivot_all.index = list(model_abbr.values())
dfrt_pivot_all.index.name = "strategy"
if showdebug: print(dfrt_pivot_all.head())

In [ ]:
if writefiles:
    dfrt_pivot_all.to_latex(buf="output/comptime_results.tex", bold_rows=True)

## Charging Comfort


In [ ]:
# execute SQL query
querycc = open("sqls/model_charging_comfort_v2.sql").read()
dfcc = pd.read_sql_query(querycc, conn)
dfcc.pvprc = dfcc.pvprc * 100
dfcc.bessprc = dfcc.bessprc * 100
if showdebug:
    print(dfcc.head())


In [ ]:
c0 = config.varpvbess_1()
c0["E_EV_MAX"]
#dfcc['SOCperc']=round(dfcc.EVSOC/c0["E_EV_MAX"],2)
dfcc['SOCperc']=round(100*dfcc.EVSOC/c0["E_EV_MAX"],0)
dfcc[dfcc.model == "stoch"]

In [ ]:
models

In [ ]:
np.array(dfcc.loc[dfcc.model == "stoch"].reset_index()[["pvprc", "bessprc", "SOCperc"]])

In [ ]:
for model in models: #list(model_desc.keys()):
    if showdebug: print("Working on model", model)
    label = model
    my_data = np.array(dfcc.loc[dfcc.model == model].reset_index()[["pvprc", "bessprc", "SOCperc"]])
    myplot = plot.contour_plot(mydata = my_data, order = 2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel = "Satisfaction [%]", color="green") #, title = model_desc[model])
    if writefiles:
        myplot.savefig("output/chargingcomfort_contour_"+label+".png",
                        bbox_inches='tight', dpi=300)
        plt.close(myplot)
    mysurf = plot.surface_plot(mydata = my_data, order = 2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel = "Satisfaction [%]", color="green") #, title = model_desc[model])
    if writefiles:
        mysurf.savefig("output/chargingcomfort_surface_"+label+".png",
                        bbox_inches='tight', dpi=300)
        plt.close(mysurf)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfcc, x="model", y="SOCperc", order=list(model_abbr.keys()),
               alpha=1, bw=1, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Satisfaction [%]")
ax.set_xticklabels(list(model_abbr.values()))

if writefiles:
    plt.savefig("output/chargingcomfort_violin.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


## SSR & SCR

In [ ]:
dfps.loc[(dfps.pvprc > 0) & (dfps.period == 0)]

In [ ]:
pvmetrics = ["SSR", "SCR"]

pvmetric_desc = {}
pvmetric_desc["SSR"] = "Self-sufficiency Rate [%]"
pvmetric_desc["SCR"] = "Self-consumption Rate [%]"


In [ ]:
np.array(dfps.loc[dfps["model"] == "stoch"].reset_index()[["pvprc", "bessprc", "SSR"]])

In [ ]:
for model in models:  # list(model_desc.keys()):
    for metric in pvmetrics:  # list(metric_desc.keys()):
        if showdebug:
            print("Working on model", model, "and metric", metric)
        label = model + "_" + metric
        my_data = np.array(dfps.loc[(dfps["model"] == model) & (dfps.period == 0) & (dfps.pvprc > 0)].reset_index()[["pvprc", "bessprc", metric]])

        if my_data[:, 2].std() == 0:
            #myplot = plot.contour_plot(mydata=my_data, order=0, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], title=model_desc[model], color="yellow")
            myplot = plot.contour_plot(mydata=my_data, order=0, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], color="yellow")
        else:
            #myplot = plot.contour_plot(mydata=my_data, order=2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], title=model_desc[model], color="yellow")
            myplot = plot.contour_plot(mydata=my_data, order=2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], color="yellow")
        if writefiles:
            myplot.savefig("output/pvmetric_contour_"+label+".png",
                           bbox_inches='tight', dpi=300)
            plt.close(myplot)
        if my_data[:, 2].std() == 0:
            #mysurf = plot.surface_plot(mydata=my_data, order=0, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], title=model_desc[model], color="yellow")
            mysurf = plot.surface_plot(mydata=my_data, order=0, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], color="yellow")
        else:
            #mysurf = plot.surface_plot(mydata=my_data, order=2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], title=model_desc[model], color="yellow")
            mysurf = plot.surface_plot(mydata=my_data, order=2, xlabel=variable_desc["pvprc"], ylabel=variable_desc["bessprc"], zlabel=pvmetric_desc[metric], color="yellow")
        if writefiles:
            mysurf.savefig("output/pvmetric_surface_"+label+".png",
                           bbox_inches='tight', dpi=300)
            plt.close(mysurf)


## Summary

In [ ]:
# label the configurations
dfps["config"] = "other"
dfps.loc[(dfps.pvprc == 0) & (dfps.bessprc == 0), "config"] = "LVG only"
dfps.loc[(dfps.pvprc == 100) & (dfps.bessprc == 0), "config"] = "LVG + PV"
dfps.loc[(dfps.pvprc == 0) & (dfps.bessprc == 100), "config"] = "LVG + BESS"
dfps.loc[(dfps.pvprc == 100) & (dfps.bessprc == 100), "config"] = "LVG + PV + BESS"
for model in dfps.model.unique():
    try:
        dfps.loc[(dfps.model == model), "model"] = model_abbr[model]
    except:
        pass
dfps

In [ ]:
dfps_pivot2 = dfps[dfps.config != "other"].pivot_table(values="GridDraw", columns=["config"], index=["model"], aggfunc=["max", "mean", "std", util.papr])
dfps_pivot2.columns.names = ["Metric", "Configuration"]
#dfps_pivot2.index = list(model_abbr.values())
dfps_pivot2.index.name = "Strategy"
if showdebug: print(dfps_pivot2.head())

In [ ]:
#models = ['direct', 'rule', 'pred', 'stoch', 'perfect']
models2 = list(model_abbr.values())
models2.reverse()
metrics2 = list(model_abbr.keys()) #['max', 'mean', 'std', 'papr']
configs = ["LVG only", "LVG + PV", "LVG + BESS", "LVG + PV + BESS"]



In [ ]:
dfps_pivot2["mean"].loc[models2,configs].round(2)

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
for metric in metrics:  # list(metric_desc.keys()):
    if showdebug:
        print("Working on metric", metric)
    label = metric
    plt.style.use("default")
    mybar3d = plot.bar3d_plot(dfps_pivot2[metric].loc[models2, configs], xlabel="Configuration", ylabel="Model", zlabel=metric_desc[metric], color="rainbow")
    if writefiles:
        mybar3d.savefig("output/model_summary_bar3d_"+label+".png",
                        bbox_inches='tight', dpi=300)
        plt.close(mybar3d)
        dfps_pivot2[metric].loc[list(model_abbr.values()), configs].round(2).to_latex(
            buf="output/model_summary_table_"+label+".tex", bold_rows=True)



In [ ]:
dfps_pivot2["max"].loc[models2[0:3],configs].mean(axis=1)


In [ ]:
dfps_pivot3 = dfps_pivot2["max"].loc[models2[0:3],configs].round(2).median(axis=1).round(2)
#dfps_pivot3["Average"] = dfps_pivot3.median(axis=1).round(2)
dfps_pivot3 = pd.DataFrame(dfps_pivot3)
dfps_pivot3.columns = ["Average"]
dfps_pivot3

In [ ]:
models3 = list(model_abbr.values())[2:]
models3


In [ ]:
dfps_pivot4 = pd.DataFrame(dfps_pivot3.loc[models3, "Average"]).T
dfps_pivot4["VSS"] = -1 * (dfps_pivot4["Stochastic"] - dfps_pivot4["Predictive"])
dfps_pivot4["EVPI"] = -1 * (dfps_pivot4["Perfect"] - dfps_pivot4["Stochastic"])
dfps_pivot4.index = ["Grid Draw Maximum [kW]"]
dfps_pivot4


In [ ]:
if writefiles:
    dfps_pivot4.to_latex(
        buf="output/model_summary_vss_evpi.tex", bold_rows=True)
